In [1]:
import bz2
import pandas as pd
import os
import shutil
from wordcloud import WordCloud

from torch.utils.data import Dataset, DataLoader

import nltk
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from torchtext.vocab import build_vocab_from_iterator

import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from collections import Counter
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch
from torch import nn

from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn import metrics

import sys
sys.path.append('../')

import contractions 

import re
import gc
import pickle



#The essentials
STOPWORDS = set(stopwords.words("english"))
print('stopwords = ', STOPWORDS)

root = os.getcwd()
datasetpath = os.path.join(root, 'dataset', 'amazon_review_ft')
assert os.path.exists(datasetpath), f'dataset path does not exist {datasetpath}'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


stopwords =  {'same', "weren't", "she's", 'were', 'not', 'mightn', 'where', 'under', 'between', 'hasn', 'shan', 'what', 'weren', 'yours', 'too', 'who', 'am', "isn't", "mustn't", 'an', 'or', "don't", 'ourselves', 'herself', 'in', 'off', 'doesn', "it's", 'below', 'after', 'the', 'is', 'while', "you'd", 'he', 'him', 'because', 'did', 'a', 'had', 'nor', "that'll", 'has', 'as', 'isn', "mightn't", 'her', 'again', "wasn't", "won't", 'have', 'wouldn', 'against', 'my', "hadn't", 'was', 'they', 'with', "didn't", 'i', 'which', 'himself', 'such', 'up', "needn't", 'ain', 'that', 'at', 'should', 'those', 'some', 'no', 'most', 'but', 'each', 'don', 'his', "hasn't", 'it', 'down', 'more', "you've", "shouldn't", 'wasn', "wouldn't", 'during', 'our', 'do', 'shouldn', 'than', 'can', "you're", 'over', 'needn', 'themselves', 'by', "haven't", 'be', 'are', 'other', 'being', "couldn't", 'aren', 'both', 'its', 'couldn', 'them', 'll', 're', 'y', 'any', 'won', 'myself', 'through', 'there', 'once', "you'll", 'how',

In [2]:
WORD_TO_IDX = os.path.join(root, 'output', 'word2idx.pkl')
IDX_TO_WORD = os.path.join(root, 'output', 'idx2word.pkl')
with open(WORD_TO_IDX, 'rb') as file:
    word2idx = pickle.load(file)
with open(IDX_TO_WORD, 'rb') as file:
    idx2word = pickle.load(file)


In [3]:
#load checkpoint after tokens np
X_TOKENS_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'X_tokens.npy')
Y_TOKENS_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'y_tokens.npy')
with open(X_TOKENS_TRAIN_OUTPUT_PATH, 'rb') as file:
    X_train = np.load(file, allow_pickle = True)
with open(Y_TOKENS_TRAIN_OUTPUT_PATH, 'rb') as file:
    y_train = np.load(file)

print("total length of dataset = ", X_train.shape)

total length of dataset =  (288000, 173)


In [4]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

In [5]:
train_dataset = TensorDataset(X_train, y_train)
print(len(train_dataset))

BATCH_SIZE = 100
train_dataloder = DataLoader(train_dataset, shuffle = True, batch_size = BATCH_SIZE)
print('train dataloader shape = ', len(train_dataloder))
sampleX, sampleY = next(iter(train_dataloder))
print("X shape = ", sampleX.shape)
print("y shape = ", sampleY.shape)

288000
train dataloader shape =  2880
X shape =  torch.Size([100, 173])
y shape =  torch.Size([100])


In [6]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, output_size, embed_size, hidden_size, num_layers, dropout_rate = 0.5):
        super(LSTMClassifier, self).__init__() 
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate 

        self.embedding = nn.Embedding(vocab_size, embedding_dim=embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, dropout = dropout_rate, batch_first = True)
        self.dropout = nn.Dropout(dropout_rate)
        self.dense = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def init_state(self, batch_size, device):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device),
                      weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device))
        return hidden
    
    def forward(self, X, state):
        batch_size, length = X.shape
        X = X.long()
        embedding = self.embedding(X)
        outputs, state = self.lstm(embedding, state)
        # print('hidden state shape = ', state[0].shape)
        # print('cell shape = ', state[1].shape)
        # print('outputs shape = ', outputs.shape)
        outputs = outputs.contiguous().view(-1, self.hidden_size)
        # print('outputs after contiguous shape = ', outputs.shape)
        outputs = self.dropout(outputs) 
        outputs = self.dense(outputs)
        outputs = self.sigmoid(outputs)
        # print('outputs after dense shape = ', outputs.shape)
        y_pred = outputs.view(batch_size, -1) 
        # print('y_pred shape = ', y_pred.shape)
        y_pred = y_pred[:, -1]
        # print('last layer y_pred shape = ', y_pred.shape)
        return y_pred, state



In [7]:
# vocab_size = len(word2idx)
# output_size =1
# embed_size = 400
# hidden_size = 500
# num_layers = 2

# test_model = LSTMClassifier(vocab_size, output_size, embed_size, hidden_size, num_layers)
# sampleX = torch.rand(10, 20)
# sampleY = torch.rand(10)

# state = test_model.init_state(sampleX.shape[0], 'cpu')
# print('hidden state shape = ', state[0].shape)
# print('cell shape = ', state[1].shape)

# sampleY_pred, state = test_model(sampleX, state)
# print('final y_pred shape = ', sampleY.shape)

# loss_fn = nn.BCELoss()

# loss = loss_fn(sampleY, sampleY)
# print(loss.item())


# Training loop

In [8]:
torch.cuda.empty_cache()
gc.collect()
# del model
# del train_dataloder

0

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using ', device)

vocab_size = len(word2idx)
print('vocab len = ', vocab_size)
output_size =1
embed_size = 100
hidden_size = 1024
num_layers = 2
GRAD_CLIP = 5
EPOCHS = 10
LR = 0.1

#clear cuda memory
torch.cuda.empty_cache()
gc.collect()

model = LSTMClassifier(vocab_size, output_size, embed_size, hidden_size, num_layers)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
loss_fn = nn.BCELoss()

model.train()
history = []

best_score = 0

CHECKPOINT_OUTPUT = os.path.join(root, 'output', 'lstm.checkpoint.torch')
HISTORY_OUTPUT = os.path.join(root, 'output', 'lstm.history.pkl')

for e in range(EPOCHS):
    state = None
    running_loss =0.0
    correct = 0
    total = 0
    loop = tqdm(train_dataloder)
    model.train()

    for X,y in loop:
        X = X.to(device)
        y = y.to(device)
        batch_size = X.shape[0]
        total += batch_size
        # DANGER: this batch size must evenly divisible by total len of dataset. 
        # else it will FAIL, configure BATCH_SIZE in dataloader
        if state == None:
            state = model.init_state(batch_size, device)

        state = [state[0].data, state[1].data]
        optimizer.zero_grad()
        y_pred, state = model(X, state)
        loss = loss_fn(y_pred, y.float())
        running_loss += loss.item()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        optimizer.step()

        #Get training accuracy
        y_pred = y_pred.round().cpu().detach().numpy()
        y = y.cpu().numpy()
        correct += (y_pred == y).sum()

        loop.set_description(f'running loss = {running_loss}, accuracy = {correct / total:.5f}')

    accuracy = correct / total

    history.append({
        'train_loss': running_loss,
        'train_acc': accuracy
    })

    if best_score < accuracy:
        torch.save(model, f'lstm.checkpoint.{accuracy:.3f}.torch')
        best_score = accuracy

    with open(HISTORY_OUTPUT, 'wb') as file:
        pickle.dump(history, file)


    # print('acc = ', correct / total)
    # print('loss value = ', running_loss)



using  cuda
vocab len =  1002


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
